In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importando os modelos que vamos testar
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Métricas de avaliação
from sklearn.metrics import accuracy_score, classification_report

# Carregamento e Análise Inicial dos Dados

In [3]:
DATASET = "../dataset/dataset_plantas_sintetico.csv"

dados = pd.read_csv(DATASET)

display(dados.sample(5))
dados.info()

,temperatura,umidade,luz,estado
158,24.99,55.72,2416,1
695,38.71,26.64,3803,0
319,35.03,27.50,3818,0
124,17.17,87.47,261,0
597,36.78,21.66,3483,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperatura  800 non-null    float64
 1   umidade      800 non-null    float64
 2   luz          800 non-null    int64  
 3   estado       800 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 25.1 KB


Como os dados foram criados via script eles já estão bem comportaos e não iremos precisar ajustar o dataset 

# Preparando dados para treinamento

In [4]:
features = ['temperatura', 'umidade', 'luz']
X = dados[features]
y = dados['estado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Utilizando GridSearch para obter melhores parametros

## Pipeline, Modelos e Grades de parametros

In [5]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    ('classifier', 'passthrough') 
])


parametros = [
    {
        'classifier': [LogisticRegression(random_state=42)], 'classifier__C': [0.1, 1.0, 10], 'classifier__solver': ['liblinear']
    },
    {
        'classifier': [DecisionTreeClassifier(random_state=42)], 'classifier__max_depth': [3, 5, 7, 10], 'classifier__criterion': ['gini', 'entropy']
    },
    {
        'classifier': [RandomForestClassifier(random_state=42)], 'classifier__n_estimators': [50, 100, 200], 'classifier__max_depth': [5, 10, None]
    },
    {
        'classifier': [SVC(random_state=42)], 'classifier__C': [0.1, 1.0, 10], 'classifier__gamma': ['scale', 'auto'], 'classifier__kernel': ['rbf', 'linear']
    }
]

## Execução

In [6]:
grid_search = GridSearchCV(pipeline, parametros, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

print("Iniciando a busca com GridSearchCV...")
grid_search.fit(X_train, y_train)
print("Busca finalizada!")

# Exibir os resultados
print(f"\nMelhor pontuação (acurácia de validação cruzada): {grid_search.best_score_:.4f}")
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

Iniciando a busca com GridSearchCV...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Busca finalizada!

Melhor pontuação (acurácia de validação cruzada): 1.0000
Melhores hiperparâmetros encontrados:
{'classifier': RandomForestClassifier(random_state=42), 'classifier__max_depth': 5, 'classifier__n_estimators': 50}


## Avaliação melhor modelo

In [7]:
modelo_grid = grid_search.best_estimator_

y_pred_grid = modelo_grid.predict(X_test)

acuracia_grid = accuracy_score(y_test, y_pred_grid)
print(f"\nAcurácia final do melhor modelo do GridSearchCV no conjunto de teste: {acuracia_grid * 100:.2f}%\n")

print("Relatório de Classificação (GridSearchCV):")
print(classification_report(y_test, y_pred_grid, target_names=['Não Saudável (0)', 'Saudável (1)']))


Acurácia final do melhor modelo do GridSearchCV no conjunto de teste: 100.00%

Relatório de Classificação (GridSearchCV):
                  precision    recall  f1-score   support

Não Saudável (0)       1.00      1.00      1.00        80
    Saudável (1)       1.00      1.00      1.00        80

        accuracy                           1.00       160
       macro avg       1.00      1.00      1.00       160
    weighted avg       1.00      1.00      1.00       160



Tivemos uma acurácia de 100% no modelo e embora no mundo real isso seria estranho como nosso dataset foi gerado de forma artificial e com regras bem definidas é esperado esse comportamento, podemos até ver o resultado dos outros modelos e veremos que boa parte deles também tiveram 100% de acuracia dada a natureza dos dados. Em um cenário real muito provavelmente a acuracia não ficaria em 100% mas ela provavelmente ficaria próxima desse valor

In [8]:
resultados = pd.DataFrame(grid_search.cv_results_)

display(resultados[['params', 'mean_test_score']].sort_values(by='mean_test_score', ascending=False)
)

,params,mean_test_score
12,{'classifier': RandomForestClassifier(random_s...,1.000000
13,{'classifier': RandomForestClassifier(random_s...,1.000000
14,{'classifier': RandomForestClassifier(random_s...,1.000000
15,{'classifier': RandomForestClassifier(random_s...,1.000000
11,{'classifier': RandomForestClassifier(random_s...,1.000000
18,{'classifier': RandomForestClassifier(random_s...,1.000000
28,"{'classifier': SVC(random_state=42), 'classifi...",1.000000
30,"{'classifier': SVC(random_state=42), 'classifi...",1.000000
24,"{'classifier': SVC(random_state=42), 'classifi...",1.000000
26,"{'classifier': SVC(random_state=42), 'classifi...",1.000000


## Salvando o melhor modelo

In [9]:
NOME_ARQUIVO_GRID = 'modelo_gridsearch.pkl'
joblib.dump(modelo_grid, NOME_ARQUIVO_GRID)

['modelo_gridsearch.pkl']

# Utilizando boosting com early stopping

## Treinamento

In [10]:
X_train_boost, X_val, y_train_boost, y_val = train_test_split(
    X_train, y_train, test_size=0.15, random_state=42, stratify=y_train
)

xgb_model = XGBClassifier(
    n_estimators=1000, learning_rate=0.1, use_label_encoder=False,
    early_stopping_rounds=10, eval_metric='logloss', random_state=42
)

xgb_model.fit(
    X_train_boost, y_train_boost,
    eval_set=[(X_val, y_val)],
    verbose=False
)

d:\Documents\Fiap - atividades\Fase 5\fase-5-cap-1\src\ir-alem\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [23:24:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,10
,enable_categorical,False
,eval_metric,'logloss'


## Avaliação

In [11]:

y_pred_xgb = xgb_model.predict(X_test)
acuracia_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"\nAcurácia do XGBoost no conjunto de teste: {acuracia_xgb * 100:.2f}%\n")

print("Relatório de Classificação do XGBoost:")
print(classification_report(y_test, y_pred_xgb, target_names=['Não Saudável (0)', 'Saudável (1)']))

NOME_ARQUIVO_XGB = 'modelo_xgboost.pkl'
joblib.dump(xgb_model, NOME_ARQUIVO_XGB)


Acurácia do XGBoost no conjunto de teste: 99.38%

Relatório de Classificação do XGBoost:
                  precision    recall  f1-score   support

Não Saudável (0)       0.99      1.00      0.99        80
    Saudável (1)       1.00      0.99      0.99        80

        accuracy                           0.99       160
       macro avg       0.99      0.99      0.99       160
    weighted avg       0.99      0.99      0.99       160



['modelo_xgboost.pkl']

# Escolhendo o melhor Modelo

Neste notebook, exploramos duas abordagens para encontrar o melhor modelo de classificação:

1.  **GridSearchCV:** Testamos sistematicamente quatro algoritmos diferentes (Regressão Logística, Árvore de Decisão, Random Forest e SVM) com uma variedade de hiperparâmetros. O melhor modelo encontrado por este método obteve uma acurácia de **100%** no conjunto de teste. O modelo vencedor foi: `RandomForestClassifier`.

2.  **XGBoost com Early Stopping:** Focamos em um único algoritmo de boosting de alta performance e usamos um conjunto de validação para otimizar o número de estimadores, evitando overfitting. Este modelo alcançou uma acurácia de **99.38%** no conjunto de teste.

**Decisão:**

Com base nos resultados, o modelo com a maior acurácia no conjunto de teste foi o **GridSearchCV**. Portanto, para a implementação final do projeto, o arquivo **'modelo_gridsearch.pkl'** será o escolhido. Ambos os modelos apresentaram excelente performance, demonstrando a clareza dos dados sintéticos gerados.